In [1]:
import pandas as pd
from subprocess import run, PIPE
from easyfinemap.utils import make_SNPID_unique
from easyfinemap import Loci
import time

In [2]:
a = pd.read_csv('~/zk_finemap/CA249.sig.leadsnp.txt', sep='\t')

In [6]:
a[a['P']<=5e-8]

,SNPID,CHR,BP,rsID,EA,NEA,BETA,SE,P,Zscore,COJO_BETA,COJO_SE,COJO_P
0,1-10790797-A-C,1,10790797,rs284277,C,A,0.0422,0.0069,1.245000e-09,6.115942,0.042200,0.006900,9.605180e-10
1,1-22282619-C-T,1,22282619,rs7529220,C,T,0.0621,0.0098,1.983000e-10,6.336735,0.062100,0.009800,2.348710e-10
2,1-41544279-G-T,1,41544279,rs2885697,G,T,0.0439,0.0070,2.884000e-10,6.271429,0.043127,0.007001,7.260070e-10
3,1-51535039-A-G,1,51535039,rs146518726,G,A,-0.1605,0.0207,8.270000e-15,-7.753623,-0.158664,0.020703,1.804100e-14
4,1-112382671-A-G,1,112382671,rs11102343,G,A,-0.0663,0.0111,2.230000e-09,-5.972973,-0.060945,0.011113,4.158360e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,18-46474192-C-T,18,46474192,rs9953366,C,T,0.0490,0.0073,1.823000e-11,6.712329,0.051302,0.007311,2.263250e-12
142,18-48679522-C-T,18,48679522,rs9963878,C,T,0.0653,0.0120,4.845000e-08,5.441667,0.069868,0.012018,6.111540e-09
143,21-36119111-G-T,21,36119111,rs2834618,G,T,-0.0944,0.0112,3.412000e-17,-8.428571,-0.094400,0.011200,3.506580e-17
144,22-18597502-C-T,22,18597502,rs464901,C,T,-0.0508,0.0072,1.531000e-12,-7.055556,-0.052315,0.007204,3.817960e-13


## test data

In [22]:
df = pd.read_csv('~/zk_finemap/normalized/GCST90132314.txt.gz', sep='\t')
df = df[df['CHR']==1]
df = make_SNPID_unique(df, replace_rsIDcol=True, remove_duplicates=True)
df.to_csv('./test_chr1.txt', sep='\t', index=False)

In [15]:
sample_size = 500000

## full chr

In [27]:
df = pd.read_csv('./test_chr1.txt', sep='\t')
inputdf = df[['rsID', 'EA', 'NEA', 'EAF', 'BETA', 'SE', 'P']].copy()
inputdf['N'] = sample_size
inputdf.columns = ['SNP', 'A1', 'A2', 'freq', 'b', 'se', 'p', 'N']
inputdf.to_csv('./test_chr1_fullchr.txt', sep='\t', index=False)

In [31]:
cmd = '''gcta64 --cojo-slct \
    --cojo-file ./test_chr1_fullchr.txt \
    --cojo-p 5e-8 --cojo-wind 10000 \
    --cojo-collinear 0.9 \
    --out ./test_chr1_fullchr \
    --bfile /beegfs/home/wangjianhua/Resources/LD_REF/hg19/plink/1000G_Phase3/EUR/EUR.valid.chr1'''
res = run(cmd, shell=True, stdout=PIPE, stderr=PIPE)

## split ld

In [47]:
df = pd.read_csv('./test_chr1.txt', sep='\t')
inputdf = df[['rsID', 'EA', 'NEA', 'EAF', 'BETA', 'SE', 'P']].copy()
inputdf['N'] = sample_size
inputdf.columns = ['SNP', 'A1', 'A2', 'freq', 'b', 'se', 'p', 'N']

In [40]:
l = Loci()

In [44]:
lead_snp, loci = l.identify_indep_loci(df, loci_extend=1500, if_merge=True, distance=1500)

In [51]:
for i in loci.index:
    inputdf = df[(df['BP']>=loci.loc[i, 'START'])&(df['BP']<=loci.loc[i, 'END'])].copy()
    inputdf = inputdf[['rsID', 'EA', 'NEA', 'EAF', 'BETA', 'SE', 'P']].copy()
    inputdf['N'] = sample_size
    inputdf.columns = ['SNP', 'A1', 'A2', 'freq', 'b', 'se', 'p', 'N']
    inputdf.to_csv(f'./test_chr1_{i}.txt', sep='\t', index=False)
    cmd = f'''gcta64 --cojo-slct \
    --cojo-file ./test_chr1_{i}.txt \
    --cojo-p 5e-8 --cojo-wind 10000 \
    --cojo-collinear 0.9 \
    --out ./test_chr1_{i} \
    --bfile /beegfs/home/wangjianhua/Resources/LD_REF/hg19/plink/1000G_Phase3/EUR/EUR.valid.chr1'''
    res = run(cmd, shell=True, stdout=PIPE, stderr=PIPE)

In [52]:
len(loci)

13

## only significant snps

In [32]:
df = pd.read_csv('./test_chr1.txt', sep='\t')
df = df[df['P']<=5e-8]
inputdf = df[['rsID', 'EA', 'NEA', 'EAF', 'BETA', 'SE', 'P']].copy()
inputdf['N'] = sample_size
inputdf.columns = ['SNP', 'A1', 'A2', 'freq', 'b', 'se', 'p', 'N']
inputdf.to_csv('./test_chr1_onlysig.txt', sep='\t', index=False)

In [33]:
cmd = '''gcta64 --cojo-slct \
    --cojo-file ./test_chr1_onlysig.txt \
    --cojo-p 5e-8 --cojo-wind 10000 \
    --cojo-collinear 0.9 \
    --out ./test_chr1_onlysig \
    --bfile /beegfs/home/wangjianhua/Resources/LD_REF/hg19/plink/1000G_Phase3/EUR/EUR.valid.chr1'''
res = run(cmd, shell=True, stdout=PIPE, stderr=PIPE)

## Compare the results

In [60]:
fullchr = pd.read_csv('./test_chr1_fullchr.jma.cojo', sep='\t')
onlysig = pd.read_csv('./test_chr1_onlysig.jma.cojo', sep='\t')
split = []
for i in loci.index:
    split.append(pd.read_csv(f'./test_chr1_{i}.jma.cojo', sep='\t'))
split = pd.concat(split, ignore_index=True)

In [61]:
fullchr.index = fullchr['SNP']
onlysig.index = onlysig['SNP']
split.index = split['SNP']
fullchr['onlysig-pJ'] = onlysig['pJ']
fullchr['onlysig-bJ'] = onlysig['bJ']
fullchr['onlysig-seJ'] = onlysig['bJ_se']
fullchr['split-pJ'] = split['pJ']
fullchr['split-bJ'] = split['bJ']
fullchr['split-seJ'] = split['bJ_se']

In [63]:
fullchr[fullchr['p']<=5e-8]

,Chr,SNP,bp,refA,freq,b,se,p,n,freq_geno,bJ,bJ_se,pJ,LD_r,onlysig-pJ,onlysig-bJ,onlysig-seJ,split-pJ,split-bJ,split-seJ
SNP,,,,,,,,,,,,,,,,,,,,
1-2245570-C-G,1,1-2245570-C-G,2245570,C,0.2800,0.041638,0.005804,7.264050e-13,895016.0,0.292247,0.049604,0.005844,2.110480e-17,0.020326,2.634250e-14,0.044261,0.005812,5.553440e-16,0.047242,0.005833
1-2985885-C-G,1,1-2985885-C-G,2985885,C,0.6118,-0.035136,0.005753,1.012640e-09,773175.0,0.611332,-0.032662,0.005777,1.567390e-08,0.061948,4.130520e-09,-0.033876,0.005762,NaN,NaN,NaN
1-3325912-A-C,1,1-3325912-A-C,3325912,A,0.1404,0.048460,0.007234,2.094810e-11,962415.0,0.162028,0.058402,0.007337,1.723930e-15,0.057439,1.489920e-11,0.048952,0.007253,2.405160e-15,0.057847,0.007305
1-27284913-C-T,1,1-27284913-C-T,27284913,T,0.0237,0.095092,0.017261,3.604290e-08,881684.0,0.024851,0.095092,0.017261,3.606160e-08,0.000000,3.607980e-08,0.095092,0.017261,3.605960e-08,0.095092,0.017261
1-38461319-A-C,1,1-38461319-A-C,38461319,A,0.5403,0.035656,0.005178,5.738790e-12,912646.0,0.532803,0.035656,0.005178,5.745860e-12,0.000000,5.752680e-12,0.035656,0.005178,5.745570e-12,0.035656,0.005178
1-55489542-C-T,1,1-55489542-C-T,55489542,T,0.2400,-0.053319,0.005837,6.592140e-20,977854.0,0.233598,-0.043085,0.005943,4.165220e-13,0.156549,9.310760e-16,-0.047423,0.005902,4.165400e-13,-0.043085,0.005943
1-55505647-G-T,1,1-55505647-G-T,55505647,T,0.0154,-0.251338,0.022653,1.324340e-28,781026.0,0.020875,-0.221092,0.022955,5.883870e-22,-0.003910,2.871360e-20,-0.211335,0.022912,5.885010e-22,-0.221092,0.022955
1-55521313-G-T,1,1-55521313-G-T,55521313,T,0.6329,0.037799,0.005202,3.713170e-13,966513.0,0.625249,0.029100,0.005258,3.118040e-08,0.060708,NaN,NaN,NaN,3.118110e-08,0.029100,0.005258
1-56877509-C-T,1,1-56877509-C-T,56877509,T,0.4712,0.036654,0.004895,7.011690e-14,1017840.0,0.527833,0.032189,0.004919,6.003420e-11,0.032547,4.458070e-13,0.035490,0.004901,6.003550e-11,0.032189,0.004919


In [64]:
len(fullchr[fullchr['p']<=5e-8])

23